# Assembly Tree Generation and Evaluation

## Formulation
 - State Space S = {Every Possible Subassembly}
 - Action Space A = {Every Connection that can currently be removed}
 - P(s'|s,a) = Feasability Checks
 - R(s,a) = Cost of removing a certain connection in line with the dynamics 
 - Graph Assumption: Order doesn't matter

## Notes
 - Next Steps:
     - Randomly generate pathing as baseline
     - Compare to DQN approach


In [35]:
import sys, random, math, time
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

## Preliminary Code (Object Classes)

# HELPER FUNCTIONS

In [36]:
def hierarchy_pos(G, root=None, width=20., vert_gap = 0.4, vert_loc = 0, xcenter = 0.5):  
    # From Joel's answer at https://stackoverflow.com/a/29597209/2966723.  
    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):   
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            children.remove(parent)  
        if len(children)!=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in children:
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos          
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)


# Generating Subassemblies in a recursive manner
def recurGen(p, H, G):
    currCon = list(H.edges())
    if(len(currCon) > 0):
        i = G.number_of_nodes() + 1
        for c in currCon:
            Hnew = H.copy()
            Hnew.remove_edges_from([c])
            if(c in seqConstraint.keys() and seqConstraint[c] in Hnew.edges()):
                pass # This means a feasability constraint has been failed!
            else:
                idString = str(Hnew.edges())
                nextState = next((x for x, y in nx.get_node_attributes(G,'idS').items() if y == idString), None)
                if(nextState is not None):
                    Rew, Hnew = R(p, c, H)
                    G.add_edge(p, nextState, a=c, r=Rew)
                else:
                    G.add_node(i, idS=idString)
                    Rew, Hnew = R(p,c,H)
                    G.add_edge(p, i, a=c, r=Rew)
                    G = recurGen(i, Hnew, G)
            i = G.number_of_nodes() + 1
        return G
    else:
        return G


# Getting next set of possible states and actions
def nextGen(H):
    currCon = list(H.edges())
    G = nx.DiGraph()
    G.add_node(1, value=H, idS=str(H.edges()))

    if(len(currCon) > 0):
        for c in currCon:
            i = G.number_of_nodes() + 1
            if(c in seqConstraint.keys() and seqConstraint[c] in Hnew.edges()):
                pass # This means a feasability constraint has been failed!
            else:
                Hnew = H.copy()
                Hnew.remove_edges_from([c])
                G.add_node(i, idS=str(Hnew.edges()))
                Rew, Hnew = R(1,c,H)
                G.add_edge(1, i, a=c, r=Rew)

                # Hnew.remove_edges_from([c])
                # G.add_node(i, idS=str(Hnew.edges()))
                # G.add_edge(1, i, a=c, r=R(1,c))
        return G
    else:
        return None
    
    
# # Checking Feasability
# def P(sN, s, a):
#     # Geometric Feasability
#     pass
#     # Sequential Constraints again


# def genRewards(assembled):
#     Rewards = {i: random.randint(0, 10) for i in list(assembled.H.edges())}
#     return Rewards

#def R(s, a):
    if((s,a) not in Rewards.keys()):
        Rewards[(s,a)] = (a[0]+a[1]) + random.randint(0, 4)
    return Rewards[(s,a)]



def R(s,a,H): # Reward Function Assuming there is a CAZ (newest numbers)
    Reward = 0
    APieces = list(nx.connected_components(H))
    Hnew = H.copy()
    Hnew.remove_edges_from([a])
    BPieces = list(nx.connected_components(Hnew))
    if(Hnew.number_of_edges() == 0): # Check if fully deconstructed
        return 1, Hnew
    elif(len(BPieces) - len(APieces) > 0):
        diffPieces = [list(i) for i in BPieces if i not in APieces and len(i) <= 3]
        # Check if removing last connection for a given part
        for i in diffPieces:
            # Check if creating multiple assemblies and sizes of these new assemblies
            if(len(i) == 1):
                if(Hnew.nodes[i[0]]["loc"] == "CL"): # Going from CL to SV
                    Hnew.nodes[i[0]]["loc"] = "SV"
                    Reward += -(0.0468 + 0.0499) #SV-CL + CL-SV
                else: # Going from CAZ to SV
                    Hnew.nodes[i[0]]["loc"] = "SV"
                    Reward += -(0.0403 + 0.0420) #SV-CAZ + CAZ-SV

            elif(len(i) == 2): # Have to fix last bit problem
                for p in i:
                    if(Hnew.nodes[p]["loc"] == "CL"): # Going from CL to CAZ
                        Hnew.nodes[p]["loc"] = "CAZ"
                Reward += -(0.0307 + 0.0415) #SV-CL + CL-CAZ2

            elif(len(i) == 3): # Have to fix last problem
                for p in i:
                    if(Hnew.nodes[p]["loc"] == "CL"): # Going from CL to CAZ
                        Hnew.nodes[p]["loc"] = "CAZ"
                Reward += -(0.0307 + 0.0475) #SV-CL + CL-CAZ3
    return Reward, Hnew

## Scenario Initialization

In [37]:
# Initialize the inputs (will later read in from file [JSON])

#define topology 
H = nx.Graph()
H.add_edge(1,2)
H.add_edge(2,3)
H.add_edge(3,5)
H.add_edge(4,5)
H.add_edge(5,6)
H.add_edge(6,7)
H.add_edge(7,8)
nx.set_node_attributes(H, "CL", "loc")

numParts = H.number_of_nodes()
numConnections = H.number_of_edges()
numActions = (numConnections-1)*pow(2,numConnections) + 1
numStates = numConnections*pow(2,numConnections-1) + 1

#SEQUENCING CONSTRAINTS (will later read in from file [JSON])
# Of the format {connection touple: connection touple that must be remove first}
seqConstraint = {}

print("CONFIGURATION: \n\t%d Parts,\n\t%d Connections in Assembly,\n\t%d Subassemblies, \n\t%d Potential Edges, \n\t%d Sequential Constraints"\
      % (numParts, numConnections, numStates, numActions, len(seqConstraint)))

CONFIGURATION: 
	8 Parts,
	7 Connections in Assembly,
	449 Subassemblies, 
	769 Potential Edges, 
	0 Sequential Constraints


## Running the Assembly Generation

In [38]:
# Simple initial initialization (will later read in from file [JSON])
Rewards = {}

# Generate the assembly tree
start = time.time()
G = nx.DiGraph()
G.add_node(1, idS=str(H.edges()))
G = recurGen(1, H, G)
end = time.time()
print("Assembly Generation Time: %f" % (end-start))

print(list(H.edges()))
print("DONE: %d Nodes and %d Edges" % (G.number_of_nodes(), G.number_of_edges()))

Assembly Generation Time: 0.058949
[(1, 2), (2, 3), (3, 5), (5, 4), (5, 6), (6, 7), (7, 8)]
DONE: 128 Nodes and 448 Edges


## Tabular Q-Learning

In [44]:
maxIter = 100
c = 0
G = nx.DiGraph()
Q = {}
ϵ = 0.3
gamma = 1
policy = {}

while c < maxIter:
    # Get random state
    currPos = H.copy()
    numCuts = random.randint(0, currPos.number_of_edges()-1)
    edges = list(currPos.edges())
    for i in range(numCuts):
        rc = edges.pop(random.randint(0, len(edges)-1))
        currPos.remove_edges_from([rc])
    # print(currPos)
    #nx.draw(currPos, with_labels=True, font_color="#FFFFFF")
    
    # Get random action
    nextPos = currPos.copy()
    edgeList = list(currPos.edges())
    if(len(edgeList) > 0):
        A = edgeList[random.randint(0, len(edgeList)-1)]
        nextPos.remove_edges_from([A])
    else:
        continue
    
    # Find "sN"s
    G.add_node(1, idS=str(nextPos.edges()))
    nearest = nextGen(nextPos)
    # print(nearest)
    if(nearest is None):
        continue
    
    # Pick best and go
    actions = nx.get_edge_attributes(nearest,'r')
    maxA = random.choice([k for k in actions.keys() if actions[k] == max(actions.values())])
    s = currPos.copy()
    sN = currPos.remove_edges_from([maxA])
    
    if((s,A) in Q.keys()):
        Qp = [0]
        for e in list(sN.edges()):
            if((sN,e) in Q.keys()):
                Qp.append(Q[sN,e])
        Q[(s,maxA)] = Q[(s,maxA)] + ϵ*(actions[maxA] + gamma*max(Qp) - Q[(s,maxA)])
    else:
        Q[(s,maxA)] = actions[maxA]
        
    c += 1

Graph with 8 nodes and 3 edges
DiGraph with 3 nodes and 2 edges
{(1, 2): -0.19340000000000002, (1, 3): -0.19340000000000002}
Graph with 8 nodes and 6 edges
DiGraph with 6 nodes and 5 edges
{(1, 2): -0.09670000000000001, (1, 3): -0.0722, (1, 4): -0.09670000000000001, (1, 5): -0.0722, (1, 6): -0.09670000000000001}
Graph with 8 nodes and 6 edges
DiGraph with 6 nodes and 5 edges
{(1, 2): -0.19340000000000002, (1, 3): -0.1749, (1, 4): -0.1749, (1, 5): -0.1749, (1, 6): -0.19340000000000002}
Graph with 8 nodes and 3 edges
DiGraph with 3 nodes and 2 edges
{(1, 2): -0.19340000000000002, (1, 3): -0.19340000000000002}
Graph with 8 nodes and 5 edges
DiGraph with 5 nodes and 4 edges
{(1, 2): -0.1689, (1, 3): -0.1689, (1, 4): -0.1689, (1, 5): -0.1689}
Graph with 8 nodes and 5 edges
DiGraph with 5 nodes and 4 edges
{(1, 2): -0.19340000000000002, (1, 3): -0.19340000000000002, (1, 4): -0.1689, (1, 5): -0.1689}
Graph with 8 nodes and 4 edges
DiGraph with 4 nodes and 3 edges
{(1, 2): -0.1749, (1, 3): -0.

AttributeError: 'NoneType' object has no attribute 'is_multigraph'

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        # 2 input channels, 1 output channels
        # kernel
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(2, 1, 1)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = QNet()
print(net)


Using cpu device
QNet(
  (conv1): Conv2d(2, 1, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Value Iteration

In [ ]:
def ValueIteration(G, H, maxIter, eps):
    S = range(1, G.number_of_nodes()+1)
    A = list(H.edges())

    V = {s: 0.0 for s in S}
    Vnew = V.copy()
    π = {s: (0, (0,0)) for s in S}
    delta = max([abs(V[s] - Vnew[s]) for s in S])

    c = 0
    while(c <= maxIter):
        delta = 0
        for s in S:
            nextStates = G[s]
            for sN in nextStates.items():
                newV = sN[1]['r'] + V[sN[0]]
                if(newV > Vnew[s]):
                    Vnew[s] = newV
                    π[s] = (sN[0], sN[1]['a'])
        delta = max([abs(V[s] - Vnew[s]) for s in S])
        if(delta < eps):
            print("Converged!")
            return Vnew, π
        V = Vnew.copy()
        c += 1
    print("Done")

maxIter = 100000
eps = 1e-6
start = time.time()
V, π = ValueIteration(G, H, maxIter, eps)
end = time.time()
print("Value Iteration Time: %f" % (end-start))

# Graphing Suite

In [ ]:
try:
    pos = hierarchy_pos(G,1)
except:
    pos = nx.planar_layout(G)
actions = nx.get_edge_attributes(G,'r')
labels = nx.get_node_attributes(G,'r')
nx.draw(G, pos, with_labels=True, font_color="#FFFFFF")
nx.draw_networkx_edge_labels(G, pos, edge_labels = actions)


try:
    S = range(1, G.number_of_nodes()+1)
    path = [1, π[1][0]]
    path_edges = [(1, path[-1])]
    while(True):
        currState = path[-1]
        nextState = π[currState][0]
        if(nextState in S):
            path.append(nextState)
            path_edges.append((currState, nextState))
        else:
            break
    print(path_edges)
    nx.draw_networkx_nodes(G,pos,nodelist=path,node_color='r')
    nx.draw_networkx_edges(G,pos,edgelist=path_edges,edge_color='r',width=1)
except:
    print("No Policy Found")

print("DONE: %d Nodes and %d Edges" % (G.number_of_nodes(), G.number_of_edges()))

In [ ]:
nx.draw(H, with_labels=True, font_color="#FFFFFF")

In [ ]:
V = G.copy()
try:
    plotVis(V)
except:
    try:
        plotVis(V)
    except:
        print("Nope!")